# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [209]:
import networkx as nx
from networkx.algorithms.flow import edmonds_karp
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [210]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [211]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [212]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    
    # test if  we have a graph where the total demand is not equal to zero . 
    test = []
    for state in graph.nodes():
        test.append(graph.node[state]['demand'])

    if sum(test) != 0:
        raise nx.NetworkXUnfeasible("An error is thrown if there is no flow satisfying the demands.") 
        
    #Create a super source S
    
    G = graph.copy()
    G.add_node('S')
    for state in G.nodes():
        if state != 'S':
            if G.node[state]['demand'] < 0:
                G.add_edge('S', state)
                G.edge['S'][state]['capacity'] = -G.node[state]['demand']
    
    #Create super sink T
    
    G.add_node('T')
    for state in G.nodes():
        if state != 'S' and state != 'T':
            if G.node[state]['demand'] > 0:
                G.add_edge(state, 'T')
                G.edge[state]['T']['capacity'] = G.node[state]['demand']
                
    
    flow_value, flow_dict = nx.maximum_flow(G,'S','T',flow_func=edmonds_karp)
  
    ## calculate total demand in the graph 
    demand = 0
    for state in G.nodes():
        if state != 'S' and state != 'T':
            if G.node[state]['demand'] > 0:
                demand += G.node[state]['demand']
    
    #Remove S and T from the flow dict of dict 
    for key in flow_dict.keys():
        if 'T' in flow_dict[key].keys():
            del flow_dict[key]['T']
        if 'S' in flow_dict[key].keys():
            del flow_dict[key]['S']
    del flow_dict['S']
    del flow_dict['T']
    
    
    #Test if the demand in the graph is equal to the max flow that was achieved by the algorithm
    if flow_value == demand:
        return flow_dict
    else:
        raise nx.NetworkXUnfeasible("An error is thrown if there is no flow satisfying the demands.")
    

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [213]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    net_flow_per_node = {k: 0 for k in flow.keys()}
    outgoing_flow_per_node = {k: 0 for k in flow.keys()}
    ingoing_flow_per_node = {k: 0 for k in flow.keys()}
    for node in flow.keys():
        outgoing_flow_per_node[node] = sum(flow[node].values())
        for node2 in flow[node].keys():
            ingoing_flow_per_node[node2] += flow[node][node2]
    for k in flow.keys():
        net_flow_per_node[k] = ingoing_flow_per_node[k] - outgoing_flow_per_node[k]
    return net_flow_per_node

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [214]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
